# Linear Algebra

## Basics

Scipp supports basic linear algebra operations in 3 dimensions using vectors of length 3 and 3x3 matrices (to define rotations).
Variables containing vectors or matrices are created using
[scipp.vectors](../generated/scipp.vectors.html#scipp-vectors) (for an array of vectors)
[scipp.vector](../generated/scipp.vector.html#scipp-vector) (for a single vector)
[scipp.matrices](../generated/scipp.matrices.html#scipp-matrices) (for and array of matrices)
[scipp.matrix](../generated/scipp.matrix.html#scipp-matrix) (for a single matrix).
Let us consider an example, creating a 1-D variable containg vectors:

In [ ]:
import scipp as sc
import numpy as np

vecs = sc.vectors(dims=['x'], unit='m', values=np.arange(2 * 3).reshape(2, 3))
vecs

While `vecs` has only a single dimension `x`, the `values` property exposes the underlying vector value as a 2-D array of `float64`:

In [ ]:
vecs.values

Access to individual vector components `x`, `y`, and `z` is provided using the special `fields` property:

In [ ]:
vecs.fields.x

These properties return a view to the underlying data and thus support setting element values as well as in-place modification:

In [ ]:
vecs.fields.x = 123.0 * sc.units.m
vecs.fields.y += 0.123 * sc.units.m
vecs.values

This works in an identical manner for matrices with the properties `xx`, `xy`, `xz`, `yx`, `yy`, `yz`, `zx`, `zy`, and `zz`.
The `values` property allows for use of, e.g., numpy functionality.
We may, e.g., compute the inverse using `numpy.linalg.inv`, which is not supported in scipp yet:

In [ ]:
np.random.seed(0)  # matrices are not singular for this seed
mats = sc.matrices(dims=['x'], values=np.random.rand(2, 3, 3))
inv = sc.matrices(dims=['x'], values=np.linalg.inv(mats.values))
(mats * inv).values.round(decimals=12)

## Example

We create some random data and use positions to create a plot with `projection='3d'` to illustrate some concepts.
Image this as an image sensor with 12x12 pixels:

In [ ]:
nx = 12
ny = 12
x = sc.linspace(dim='x', start=-0.1, stop=0.1, num=nx, unit='m')
y = sc.linspace(dim='y', start=-0.1, stop=0.1, num=ny, unit='m')
da = sc.DataArray(
    data=sc.array(dims=['x','y'], values=np.random.rand(nx, ny)),
    coords={'position':sc.geometry.position(x, y, 0.0*sc.units.m)}
)
sc.plot(da, projection="3d", positions="position")

We can use the vector-element access properties in combination with slicing to offset some of the pixels:

In [ ]:
da.coords['position']['x',5:].fields.x += 0.1 * sc.units.m
da.coords['position']['y',5:].fields.y += 0.05 * sc.units.m

sc.plot(da, projection="3d", positions="position")

We use `sc.matrix` to create a rotation matrix (in this case to rotate by 30 deg around the `y` axis) and apply it to the positions:

In [ ]:
rotation = sc.matrix(value=[[ 0.8660254, 0.0000000, 0.5000000],
                            [ 0.0000000, 1.0000000, 0.0000000],
                            [-0.5000000, 0.0000000, 0.8660254]])

da.coords['position']['x',5:] = rotation * da.coords['position']['x',5:]
sc.plot(da, projection="3d", positions="position")

<div class="alert alert-info">

**Note:** More convenient tools for creating rotations around axes are on the roadmap.

</div>